# Simon's algorithm

Simon's algorithm is closely related in principle to the Bernstein-Vazirani algorithm and the Deutsch-Josza algorithm, (which are also explained in other notebooks in this repository).

We are given an "oracle" and promised that its outputs are either **one-to-one** or **two-to-one**, i.e., it either has **exactly** as many possible outputs as possible inputs or **exactly half** as many possible outputs as possible inputs. In effect, all two-to-one functions can be thought of as being based on a "**hidden string**" encoded into the oracle, whereas one-to-one functions are the only case where the hidden string is "**0**." Our problem is therefore to determine the "hidden string" encoded into the oracle.

A classical computer, on average, needs to check about half the possible inputs to answer the question of what the hidden string is; a quantum computer needs **exponentially fewer** queries of the oracle to determine the same answer.

(...So the conventional disciplinary thinking goes, at least. But notice that we are doing 120 qubits of Simon's algorithm below, and we could argue we only need a single query, with PyQrack!)

In [1]:
import os
import random
from collections import Counter
from pyqrack import QrackSimulator

# Tell Qrack that it can use as many stabilizer qubits as it wants,
# without worrying about converting to ket for GPU:
os.environ['QRACK_MAX_CPU_QB']='-1'

# Range is up to 2 ** input_size - 1
input_size = 60

# Arbitrary value within range:
hidden_bits = 2**32 - 3

num_qubits = input_size * 2
oracle_qubits = [*range(input_size)]

# Swapping the bits randomly doesn't seem to work, for now:
oracle_swap_tuples = []
for i in range(input_size // 2):
    i = random.randint(input_size, (input_size * 2) - 1)
    j = random.randint(input_size, (input_size * 2) - 1)
    if i != j:
        oracle_swap_tuples.append((i, j))

You may change the `hidden_bits` above to be any value that can fit within the oracle register width. Your `hidden_bits` corresponds to the oracle below:

In [2]:
def oracle(sim):
    # The first portion of this oracle is effectively a one-to-one oracle,
    # in itself, (though not the only possible one-to-one oracle).
    for i in oracle_qubits:
        sim.mcx([i], i + input_size)

    # The second portion of this oracle makes it two-to-one,
    # (unless "hidden_bits" is 0):
    for i in oracle_qubits:
        # We pick any one of the "set" hidden bits:
        if (hidden_bits >> i) & 1:
            for j in oracle_qubits:
                # Act CNOT between the "set" hidden bit in input and all hidden bits in output:
                if (hidden_bits >> j) & 1:
                    sim.mcx([i], j + input_size)
            # One bit in the "i" loop is sufficient:
            break;

    # We can now swap at random in the output register
    # and still satisfy our promises about the oracle:
    for i in oracle_swap_tuples:
         sim.swap(i[0], i[1])

This is the algorithm itself. (A classical simulator of quantum computers can run the "unitary preamble" exactly once and then secondarily "sample" the measurement distribution of the end state.)

In [3]:
# Prepare the initial register state:
sim = QrackSimulator(num_qubits, isCpuGpuHybrid=False, isOpenCL=False)
for i in oracle_qubits:
    sim.h(i)

# Make exactly one query to the oracle:
oracle(sim)

# Finish the unitary portion of the algorithm, with the result from the oracle...
# (It is often presented that measurement of the second register needs to occur before this step;
# this is actually wholly theoretically unnecessary in the ideal, due to locality of quantum information.)
for i in oracle_qubits:
    sim.h(i)

# The cost in "shot" count is linear, at this point.
results = Counter(sim.measure_shots(oracle_qubits, 32768))

The ultimate point of the algorithm is that, by solving the system of equations below, **where the value of `hidden_string` is an algebraic unknown**, we can determine the value of `hidden_string`, with a probabilistic likelihood that converges exponentially faster than a "classical" algorithm, in the number of measurement "shots." (In a real hardware quantum computer, the number of "shots" would correspond to exactly the number of queries of the oracle).

Let's check if our implied system of equations is right:

In [4]:
for key in results.keys():
    x = (bin(hidden_bits & key).count("1")) & 1
    if x > 0:
        print("Failed!", hidden_bits, ".", key, "=", x, "(mod 2)")
        break
    print(hidden_bits, ".", key, "=", x, "(mod 2)")

4294967293 . 3939473839 = 0 (mod 2)
4294967293 . 1828692597 = 0 (mod 2)
4294967293 . 3473398030 = 0 (mod 2)
4294967293 . 3653470092 = 0 (mod 2)
4294967293 . 3526924244 = 0 (mod 2)
4294967293 . 1402264915 = 0 (mod 2)
4294967293 . 4229322556 = 0 (mod 2)
4294967293 . 2703798823 = 0 (mod 2)
4294967293 . 1908337186 = 0 (mod 2)
4294967293 . 3429515194 = 0 (mod 2)
4294967293 . 1430602466 = 0 (mod 2)
4294967293 . 2439075804 = 0 (mod 2)
4294967293 . 1681032119 = 0 (mod 2)
4294967293 . 541666299 = 0 (mod 2)
4294967293 . 575994664 = 0 (mod 2)
4294967293 . 979408801 = 0 (mod 2)
4294967293 . 1882014691 = 0 (mod 2)
4294967293 . 3864789132 = 0 (mod 2)
4294967293 . 4107608297 = 0 (mod 2)
4294967293 . 3289332138 = 0 (mod 2)
4294967293 . 2905192738 = 0 (mod 2)
4294967293 . 2500949429 = 0 (mod 2)
4294967293 . 2208940926 = 0 (mod 2)
4294967293 . 4132126008 = 0 (mod 2)
4294967293 . 2992536815 = 0 (mod 2)
4294967293 . 3577992550 = 0 (mod 2)
4294967293 . 3917594266 = 0 (mod 2)
4294967293 . 3077040659 = 0 (mo

4294967293 . 2020880439 = 0 (mod 2)
4294967293 . 2587839534 = 0 (mod 2)
4294967293 . 3528997920 = 0 (mod 2)
4294967293 . 1858300176 = 0 (mod 2)
4294967293 . 599735199 = 0 (mod 2)
4294967293 . 43134276 = 0 (mod 2)
4294967293 . 3353762484 = 0 (mod 2)
4294967293 . 3388605603 = 0 (mod 2)
4294967293 . 3831522630 = 0 (mod 2)
4294967293 . 903050532 = 0 (mod 2)
4294967293 . 1633399277 = 0 (mod 2)
4294967293 . 1604750091 = 0 (mod 2)
4294967293 . 3017847480 = 0 (mod 2)
4294967293 . 1833397847 = 0 (mod 2)
4294967293 . 1935357502 = 0 (mod 2)
4294967293 . 136656785 = 0 (mod 2)
4294967293 . 2161248583 = 0 (mod 2)
4294967293 . 556405329 = 0 (mod 2)
4294967293 . 3889453000 = 0 (mod 2)
4294967293 . 879470382 = 0 (mod 2)
4294967293 . 1032715721 = 0 (mod 2)
4294967293 . 832515543 = 0 (mod 2)
4294967293 . 1717723288 = 0 (mod 2)
4294967293 . 1568814731 = 0 (mod 2)
4294967293 . 3401850959 = 0 (mod 2)
4294967293 . 502168945 = 0 (mod 2)
4294967293 . 903627014 = 0 (mod 2)
4294967293 . 1777360465 = 0 (mod 2)
42